## Analýza zamestnancov VŠ

In [1]:
import pandas as pd

from pandasql import sqldf
# from sklearn import datasets

In [2]:
df = pd.read_csv('../data/preprocessed/regzam.csv', index_col=0)
display(df)

,Employee_ID,GivenName,FamilyName,DegreePedagogicalAcademic,DegreeMaster,DegreeDoctor,Employment_Workplace_University,Employment_Type,Employment_Subtype,Employment_BeginDate,Employment_Ratio,Guarantee_StudyProgram,Guarantee_University,Guarantee_Faculty,Guarantee_FormOfStudy,Guarantee_LevelOfStudy,Employment_Workplace_Faculty,Employment_EndDate
0,18799121690,Štefan,Kočan,Doc.,JUDr.,PhD.,Vysoká škola bezpečnostného manažérstva v Koši...,Docent,s titulom docent a vysokoškolským vzdelaním tr...,2020-10-01,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,14002860472,Dušan,Korgo,DOC,JUDR,PHD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,17922733804,Ján,Buzalka,prof.,PhDr.,CSc.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,16290288493,Jozef,Stieranka,prof.,Ing.,PhD.,Akadémia Policajného zboru,Profesor,s titulom profesor a vysokoškolským vzdelaním ...,2013-11-01,100,"['bezpečnostnoprávna ochrana osôb a majetku', ...","['Akadémia Policajného zboru', 'Akadémia Polic...","[None, None, None, None, None, None, None]","['1', '2', '1', '2', '2', '1', '2']","['1', '1', '2', '2', '2', '3', '3']",NaN,NaN
4,21401303068,Ľuboš,Wäldl,doc.,JUDr.,PhD.,Akadémia Policajného zboru,Profesor,s titulom docent a vysokoškolským vzdelaním tr...,2008-09-01,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32725,12110964830,Joža,Spurný,doc.,JUDr.PhDr.,Ph.D.,Vysoká škola zdravotníctva a sociálnej práce s...,Docent,s titulom docent a vysokoškolským vzdelaním tr...,2022-09-30,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32726,13500390262,Marcela,Harčárová,NaN,Mgr.,NaN,Vysoká škola zdravotníctva a sociálnej práce s...,Asistent,s vysokoškolským vzdelaním druhého stupňa,2022-09-01,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32727,17108478442,Jana,Juhásová,NaN,Mgr.,NaN,Vysoká škola zdravotníctva a sociálnej práce s...,Asistent,s vysokoškolským vzdelaním druhého stupňa,2022-09-01,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32728,15917135755,Miriama,Kubušová,NaN,Mgr.,NaN,Vysoká škola zdravotníctva a sociálnej práce s...,Asistent,s vysokoškolským vzdelaním druhého stupňa,2022-09-01,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
id_set = set(df["Employee_ID"])
print(len(id_set))

32730


-> máme Dataset s 32730 riadkami, pričom žiadne 2 Employee_ID nie sú rovnaké

In [4]:
name_set = set(df["FamilyName"] + " " + df["GivenName"])
len(name_set)

31639

-> 31639 rôznych mien; zostáva zistiť, či pri duplikátnych menách sa jedná o tú istú osobu, alebo nie.
Tiež by nás mohlo zaujímať, ako ukladali osoby, ktorým sa zmenilo meno...

In [5]:
def find_name_duplicates(df):
    name_set = set()
    name_duplicates = set()
    duplicates_ids = set()
    for index, row in df.iterrows():
        name = row["FamilyName"] + " " + row["GivenName"]
        if name in name_set:
            name_duplicates.add(name)
        else:
            name_set.add(name)
    for index, row in df.iterrows():
        name = row["FamilyName"] + " " + row["GivenName"]
        if name in name_duplicates:
            duplicates_ids.add(row["Employee_ID"])
    return tuple(name_duplicates), tuple(duplicates_ids)

In [6]:
name_duplicates, ids = find_name_duplicates(df)

In [7]:
q = f"SELECT * FROM df WHERE Employee_ID IN {ids}"
duplicates_df = sqldf(q, globals()).sort_values(by=['FamilyName', 'GivenName'])
display(duplicates_df)

,Employee_ID,GivenName,FamilyName,DegreePedagogicalAcademic,DegreeMaster,DegreeDoctor,Employment_Workplace_University,Employment_Type,Employment_Subtype,Employment_BeginDate,Employment_Ratio,Guarantee_StudyProgram,Guarantee_University,Guarantee_Faculty,Guarantee_FormOfStudy,Guarantee_LevelOfStudy,Employment_Workplace_Faculty,Employment_EndDate
1327,49910822214,Katarína,Adamíková,None,Mgr.,None,None,None,None,None,None,None,None,None,None,None,None,None
1576,18913486096,Katarína,Adamíková,None,Mgr.,None,None,None,None,None,None,None,None,None,None,None,None,None
114,39501978150,František,Adamčík,prof.,Ing.,CSc.,Technická univerzita v Košiciach,Profesor,s titulom profesor a vysokoškolským vzdelaním ...,2005-02-01,100,"['prevádzka lietadiel', 'prevádzka lietadiel',...","['Technická univerzita v Košiciach', 'Technick...","['Letecká fakulta', 'Letecká fakulta', 'Leteck...","['1', '1', '2']","['2', '3', '3']",Letecká fakulta,None
1325,17240205086,František,Adamčík,None,Ing.,PhD.,None,None,None,None,None,None,None,None,None,None,None,None
133,51642863772,Jana,Antalová,None,MUDr.,PhD.,Univerzita Komenského v Bratislave,Odborný asistent,s vysokoškolským vzdelaním tretieho stupňa,2020-08-15,100,None,None,None,None,None,Lekárska fakulta,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1876,15552653387,Tomáš,Žilka,None,MUDr.,None,Slovenská zdravotnícka univerzita v Bratislave,Zamestnanec vykonávajúci činnosti VŠ učiteľa m...,s vysokoškolským vzdelaním druhého stupňa,2022-02-01,100,None,None,None,None,None,Lekárska fakulta,None
1107,10056354153,Viktória,Žilková,None,Mgr.,None,None,None,None,None,None,None,None,None,None,None,None,None
1994,14058627798,Viktória,Žilková,None,None,MSc.,Univerzita Komenského v Bratislave,Výskumný pracovník - výskumník,s vysokoškolským vzdelaním druhého stupňa,2022-06-15,50,None,None,None,None,None,Lekárska fakulta,None
530,14639882132,Anna,Žuffová,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [8]:
df.query("GivenName == 'Viktória' and FamilyName == 'Žilková'")

,Employee_ID,GivenName,FamilyName,DegreePedagogicalAcademic,DegreeMaster,DegreeDoctor,Employment_Workplace_University,Employment_Type,Employment_Subtype,Employment_BeginDate,Employment_Ratio,Guarantee_StudyProgram,Guarantee_University,Guarantee_Faculty,Guarantee_FormOfStudy,Guarantee_LevelOfStudy,Employment_Workplace_Faculty,Employment_EndDate
17361,10056354153,Viktória,Žilková,NaN,Mgr.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32149,14058627798,Viktória,Žilková,NaN,NaN,MSc.,Univerzita Komenského v Bratislave,Výskumný pracovník - výskumník,s vysokoškolským vzdelaním druhého stupňa,2022-06-15,50,NaN,NaN,NaN,NaN,NaN,Lekárska fakulta,NaN


... V tomto bode sme zistili, že naše stiahnuté xml zrejme obsahuje len dáta pre aktuálnych zamestnancov...

Pre minulých (zamestnancov) neobsahuje údaje o zamestnaní, predovšetkým nevieme z dát zistiť, v akom období a kde boli zamestnaní, hoci ich meno je v dátach.

In [9]:
df.query("Employment_EndDate.isna()")

,Employee_ID,GivenName,FamilyName,DegreePedagogicalAcademic,DegreeMaster,DegreeDoctor,Employment_Workplace_University,Employment_Type,Employment_Subtype,Employment_BeginDate,Employment_Ratio,Guarantee_StudyProgram,Guarantee_University,Guarantee_Faculty,Guarantee_FormOfStudy,Guarantee_LevelOfStudy,Employment_Workplace_Faculty,Employment_EndDate
0,18799121690,Štefan,Kočan,Doc.,JUDr.,PhD.,Vysoká škola bezpečnostného manažérstva v Koši...,Docent,s titulom docent a vysokoškolským vzdelaním tr...,2020-10-01,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,14002860472,Dušan,Korgo,DOC,JUDR,PHD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,17922733804,Ján,Buzalka,prof.,PhDr.,CSc.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,16290288493,Jozef,Stieranka,prof.,Ing.,PhD.,Akadémia Policajného zboru,Profesor,s titulom profesor a vysokoškolským vzdelaním ...,2013-11-01,100,"['bezpečnostnoprávna ochrana osôb a majetku', ...","['Akadémia Policajného zboru', 'Akadémia Polic...","[None, None, None, None, None, None, None]","['1', '2', '1', '2', '2', '1', '2']","['1', '1', '2', '2', '2', '3', '3']",NaN,NaN
4,21401303068,Ľuboš,Wäldl,doc.,JUDr.,PhD.,Akadémia Policajného zboru,Profesor,s titulom docent a vysokoškolským vzdelaním tr...,2008-09-01,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32725,12110964830,Joža,Spurný,doc.,JUDr.PhDr.,Ph.D.,Vysoká škola zdravotníctva a sociálnej práce s...,Docent,s titulom docent a vysokoškolským vzdelaním tr...,2022-09-30,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32726,13500390262,Marcela,Harčárová,NaN,Mgr.,NaN,Vysoká škola zdravotníctva a sociálnej práce s...,Asistent,s vysokoškolským vzdelaním druhého stupňa,2022-09-01,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32727,17108478442,Jana,Juhásová,NaN,Mgr.,NaN,Vysoká škola zdravotníctva a sociálnej práce s...,Asistent,s vysokoškolským vzdelaním druhého stupňa,2022-09-01,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32728,15917135755,Miriama,Kubušová,NaN,Mgr.,NaN,Vysoká škola zdravotníctva a sociálnej práce s...,Asistent,s vysokoškolským vzdelaním druhého stupňa,2022-09-01,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## -> Až 32434 riadkov nemá EndDate, čo aj potvrdzuje našu hypotézu, že máme zlé dáta :(  (iba súčasné úväzky)

## **- Pracujú zamestnanci aj na viacerých pracoviskách a vidno v tom nejaké trendy alebo zhluky?**

Vyberieme iba dáta o súčasných zamestnancoch, t.j. odfiltrujeme tie riadky, ktorým chýba BeginDate

In [34]:
current_employees = df[df['Employment_BeginDate'].notna()]
display(current_employees)

,Employee_ID,GivenName,FamilyName,DegreePedagogicalAcademic,DegreeMaster,DegreeDoctor,Employment_Workplace_University,Employment_Type,Employment_Subtype,Employment_BeginDate,Employment_Ratio,Guarantee_StudyProgram,Guarantee_University,Guarantee_Faculty,Guarantee_FormOfStudy,Guarantee_LevelOfStudy,Employment_Workplace_Faculty,Employment_EndDate
0,18799121690,Štefan,Kočan,Doc.,JUDr.,PhD.,Vysoká škola bezpečnostného manažérstva v Koši...,Docent,s titulom docent a vysokoškolským vzdelaním tr...,2020-10-01,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,16290288493,Jozef,Stieranka,prof.,Ing.,PhD.,Akadémia Policajného zboru,Profesor,s titulom profesor a vysokoškolským vzdelaním ...,2013-11-01,100,"['bezpečnostnoprávna ochrana osôb a majetku', ...","['Akadémia Policajného zboru', 'Akadémia Polic...","[None, None, None, None, None, None, None]","['1', '2', '1', '2', '2', '1', '2']","['1', '1', '2', '2', '2', '3', '3']",NaN,NaN
4,21401303068,Ľuboš,Wäldl,doc.,JUDr.,PhD.,Akadémia Policajného zboru,Profesor,s titulom docent a vysokoškolským vzdelaním tr...,2008-09-01,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,69828748660,Matej,Kostrec,NaN,JUDr.,PhD.,Akadémia Policajného zboru,Odborný asistent,s vysokoškolským vzdelaním tretieho stupňa,2015-01-01,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,91221451321,Magdaléna,Krajníková,NaN,JUDr.,PhD.,Akadémia Policajného zboru,Odborný asistent,s vysokoškolským vzdelaním tretieho stupňa,2013-04-01,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32725,12110964830,Joža,Spurný,doc.,JUDr.PhDr.,Ph.D.,Vysoká škola zdravotníctva a sociálnej práce s...,Docent,s titulom docent a vysokoškolským vzdelaním tr...,2022-09-30,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32726,13500390262,Marcela,Harčárová,NaN,Mgr.,NaN,Vysoká škola zdravotníctva a sociálnej práce s...,Asistent,s vysokoškolským vzdelaním druhého stupňa,2022-09-01,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32727,17108478442,Jana,Juhásová,NaN,Mgr.,NaN,Vysoká škola zdravotníctva a sociálnej práce s...,Asistent,s vysokoškolským vzdelaním druhého stupňa,2022-09-01,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32728,15917135755,Miriama,Kubušová,NaN,Mgr.,NaN,Vysoká škola zdravotníctva a sociálnej práce s...,Asistent,s vysokoškolským vzdelaním druhého stupňa,2022-09-01,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN


- Kontrola, či majú všetci vybraní zamestnanci EndDate NaN:

In [37]:
current_employees[current_employees['Employment_EndDate'].notna()]

,Employee_ID,GivenName,FamilyName,DegreePedagogicalAcademic,DegreeMaster,DegreeDoctor,Employment_Workplace_University,Employment_Type,Employment_Subtype,Employment_BeginDate,Employment_Ratio,Guarantee_StudyProgram,Guarantee_University,Guarantee_Faculty,Guarantee_FormOfStudy,Guarantee_LevelOfStudy,Employment_Workplace_Faculty,Employment_EndDate
260,18814294599,Zuzana,Francová,doc.,Ing.,PhD.,Ekonomická univerzita v Bratislave,Docent,s vysokoškolským vzdelaním tretieho stupňa,2022-09-01,100,NaN,NaN,NaN,NaN,NaN,Obchodná fakulta,2022-09-30
312,35831820319,Róbert,Rehák,doc.,Ing.,PhD.,Ekonomická univerzita v Bratislave,Docent,s vysokoškolským vzdelaním tretieho stupňa,2022-09-01,100,NaN,NaN,NaN,NaN,NaN,Obchodná fakulta,2022-09-30
423,10737086394,Renáta,Pakšiová,prof.,Ing. Mgr.,PhD.,Ekonomická univerzita v Bratislave,Profesor,s titulom docent a vysokoškolským vzdelaním tr...,2021-09-01,100,NaN,NaN,NaN,NaN,NaN,Fakulta hospodárskej informatiky,2022-09-30
550,20667634782,Tomáš,Dudáš,Doc.,PhD.,Ing.,Paneurópska vysoká škola,Docent,s titulom docent a vysokoškolským vzdelaním tr...,2012-05-01,100,"['ekonomika a manažment podnikania', 'ekonomik...","['Paneurópska vysoká škola', 'Paneurópska vyso...","['Fakulta ekonómie a podnikania', 'Fakulta eko...","['2', '1', '1', '2']","['1', '1', '3', '3']",Fakulta ekonómie a podnikania,2022-09-30
604,86267871612,Danuša,Lišková,NaN,NaN,NaN,Univerzita sv. Cyrila a Metoda v Trnave,Zamestnanec vykonávajúci činnosti VŠ učiteľa m...,bez vysokoškolského vzdelania,2022-09-30,105,NaN,NaN,NaN,NaN,NaN,Filozofická fakulta,2022-09-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32561,12242815951,Viera,Jakušová,NaN,Mgr.,PhD.,Univerzita Komenského v Bratislave,Odborný asistent,s vysokoškolským vzdelaním tretieho stupňa,2022-07-07,50,NaN,NaN,NaN,NaN,NaN,Právnická fakulta,2022-09-30
32578,11842718393,Patrícia,Dragunová,NaN,MUDr.,NaN,Univerzita Komenského v Bratislave,Asistent,s vysokoškolským vzdelaním druhého stupňa,2022-08-01,90,NaN,NaN,NaN,NaN,NaN,Lekárska fakulta,2022-09-30
32598,56924446213,Vratislav,Zervan,NaN,NaN,NaN,Univerzita sv. Cyrila a Metoda v Trnave,Zamestnanec vykonávajúci činnosti VŠ učiteľa m...,bez vysokoškolského vzdelania,2022-09-30,70,NaN,NaN,NaN,NaN,NaN,Filozofická fakulta,2022-09-30
32642,97138493313,Radko,Dvorščák,NaN,Ing.,NaN,Prešovská univerzita v Prešove,Zamestnanec vykonávajúci činnosti VŠ učiteľa m...,bez vysokoškolského vzdelania,2022-09-22,7.78,NaN,NaN,NaN,NaN,NaN,Fakulta športu,2022-09-30


-> nemajú, sú tam aj zamestnanci, ktorí skončili k 30.9.2022

In [43]:
set(current_employees['Employment_EndDate'])

{'2022-09-30', "['2022-09-30', '2022-09-30']", nan}

-> nechceme ani tých s EndDate 30.9.2022

In [49]:
current_employees = current_employees[current_employees['Employment_EndDate'].isna()].reset_index()

- Majú všetci University vyplnenú?

In [53]:
current_employees[current_employees['Employment_Workplace_University'].isna()]

,index,Employee_ID,GivenName,FamilyName,DegreePedagogicalAcademic,DegreeMaster,DegreeDoctor,Employment_Workplace_University,Employment_Type,Employment_Subtype,Employment_BeginDate,Employment_Ratio,Guarantee_StudyProgram,Guarantee_University,Guarantee_Faculty,Guarantee_FormOfStudy,Guarantee_LevelOfStudy,Employment_Workplace_Faculty,Employment_EndDate


-> áno.

Existujú ľudia s viacerými záznamami? Napr., existuje viacero riadkov s rovnakým indexom?

In [56]:
len(set(current_employees['Employee_ID']))

13735

=> Nie, každý riadok má svoj vlastný Employee_ID

In [84]:
counts = current_employees[['Employee_ID', 'GivenName', 'FamilyName']].groupby(['GivenName', 'FamilyName']).count().rename(columns={'Employee_ID': 'count'}).reset_index()
display(counts)

,GivenName,FamilyName,count
0,Vasil,Gluchman,1
1,Aaron,McCauley,1
2,Abdolreza,Majidi,1
3,Adam,Antl,1
4,Adam,Bžoch,1
...,...,...,...
13500,Žofia,Bárcziová,1
13501,Žofia,Dubová,1
13502,Žofia,Kuzevičová,1
13503,Žofia,Nižnanská,1


-> títo ľudia majú viacero záznamov alebo rovnaké mená:

In [87]:
counts[counts['count'] != 1]

,GivenName,FamilyName,count
187,Alena,Nagyová,2
314,Alexandra,Valenčáková,2
448,Andrea,Gažová,2
497,Andrea,Miškufová,2
546,Andrea,Vargová,2
...,...,...,...
13014,Zuzana,Štefunková,2
13165,Ľubomír,Straka,2
13238,Ľuboš,Gajdoš,2
13331,Ľudovít,Danihel,2


In [90]:
multi_employees = counts[counts['count'] != 1]
multi_employees[['GivenName', 'FamilyName']]

,GivenName,FamilyName
187,Alena,Nagyová
314,Alexandra,Valenčáková
448,Andrea,Gažová
497,Andrea,Miškufová
546,Andrea,Vargová
...,...,...
13014,Zuzana,Štefunková
13165,Ľubomír,Straka
13238,Ľuboš,Gajdoš
13331,Ľudovít,Danihel


In [86]:
current_employees[current_employees['FamilyName'] == 'Výrost']

,index,Employee_ID,GivenName,FamilyName,DegreePedagogicalAcademic,DegreeMaster,DegreeDoctor,Employment_Workplace_University,Employment_Type,Employment_Subtype,Employment_BeginDate,Employment_Ratio,Guarantee_StudyProgram,Guarantee_University,Guarantee_Faculty,Guarantee_FormOfStudy,Guarantee_LevelOfStudy,Employment_Workplace_Faculty,Employment_EndDate
324,704,53060727347,Tomáš,Výrost,doc.,Ing.,PhD.,"['Ekonomická univerzita v Bratislave', 'Techni...","['Docent', 'Docent']",['s titulom docent a vysokoškolským vzdelaním ...,"['2021-03-01', '2021-01-04']","['100', '50']",NaN,NaN,NaN,NaN,NaN,"['Obchodná fakulta', 'Ekonomická fakulta']",NaN


## Ešte raz z upravených dát:
    

In [110]:
df_workplaces = pd.read_csv('../data/preprocessed/regzam_workplaces.csv', index_col=0)
display(df_workplaces)

,Employee_ID,FamilyName,GivenName,DegreePedagogicalAcademic,DegreeMaster,DegreeDoctor,Employment_Workplace_University,Employment_Workplace_Faculty,Employment_Type,Employment_Ratio
0,18799121690,Kočan,Štefan,Doc.,JUDr.,PhD.,Vysoká škola bezpečnostného manažérstva v Koši...,NaN,Docent,100.00
1,14002860472,Korgo,Dušan,DOC,JUDR,PHD,NaN,NaN,NaN,NaN
2,17922733804,Buzalka,Ján,prof.,PhDr.,CSc.,NaN,NaN,NaN,NaN
3,16290288493,Stieranka,Jozef,prof.,Ing.,PhD.,Akadémia Policajného zboru,NaN,Profesor,100.00
4,21401303068,Wäldl,Ľuboš,doc.,JUDr.,PhD.,Akadémia Policajného zboru,NaN,Profesor,100.00
...,...,...,...,...,...,...,...,...,...,...
33358,17628642100,Peter,Fabo,NaN,RNDr.,PhD.,Žilinská univerzita v Žiline,NaN,Výskumný pracovník - výskumník,20.00
33359,17628642100,Peter,Fabo,NaN,RNDr.,PhD.,Trenčianska univerzita Alexandra Dubčeka v Tre...,Fakulta špeciálnej techniky,Výskumný pracovník - výskumník,100.00
33360,28653015119,Marianna,Berinšterová,NaN,Mgr.,PhD.,Prešovská univerzita v Prešove,Fakulta humanitných a prírodných vied,Odborný asistent,100.00
33361,28653015119,Marianna,Berinšterová,NaN,Mgr.,PhD.,Vysoká škola medzinárodného podnikania ISM Slo...,NaN,Zamestnanec vykonávajúci činnosti VŠ učiteľa m...,5.00
